In [ ]:
import random
import time

# --- ENUM FOR PIECES ---
# Positive values = White, Negative values = Black
class Piece:
    EMPTY = 0
    WP = 1
    WN = 2
    WB = 3
    WR = 4
    WQ = 5
    WK = 6
    BP = -1
    BN = -2
    BB = -3
    BR = -4
    BQ = -5
    BK = -6

# --- INITIAL BOARD STATE ---
board = [
    Piece.WR, Piece.WN, Piece.WB, Piece.WQ, Piece.WK, Piece.WB, Piece.WN, Piece.WR,  # Rank 1
    Piece.WP, Piece.WP, Piece.WP, Piece.WP, Piece.WP, Piece.WP, Piece.WP, Piece.WP,  # Rank 2
    *([Piece.EMPTY] * 16),  # Ranks 3-4
    *([Piece.EMPTY] * 16),  # Ranks 5-6
    Piece.BP, Piece.BP, Piece.BP, Piece.BP, Piece.BP, Piece.BP, Piece.BP, Piece.BP,  # Rank 7
    Piece.BR, Piece.BN, Piece.BB, Piece.BQ, Piece.BK, Piece.BB, Piece.BN, Piece.BR   # Rank 8
]

turn = 1  # 1 = White, -1 = Black

# --- MOVE STRUCT ---
class Move:
    def __init__(self, fromSquare, toSquare, capturedPiece=Piece.EMPTY):
        self.fromSquare = fromSquare
        self.toSquare = toSquare
        self.capturedPiece = capturedPiece

# --- UTILITY FUNCTIONS ---
def getRank(square): return square // 8
def getFile(square): return square % 8
def getSquareIndex(rank, file): return rank * 8 + file
def isOnBoard(square): return 0 <= square < 64

def isOpponent(pieceColor, targetSquare):
    target = board[targetSquare]
    if target == Piece.EMPTY: return False
    return (target > 0 and pieceColor < 0) or (target < 0 and pieceColor > 0)

def isSameColor(pieceColor, targetSquare):
    target = board[targetSquare]
    if target == Piece.EMPTY: return False
    return (target > 0 and pieceColor > 0) or (target < 0 and pieceColor < 0)

# --- PIECE VALUE ---
def pieceValue(piece):
    values = {
        Piece.WP: 100, Piece.WN: 300, Piece.WB: 300,
        Piece.WR: 500, Piece.WQ: 900, Piece.WK: 20000
    }
    return values.get(abs(piece), 0)

# --- PRINT BOARD ---
def printBoard():
    pieceChar = {
        Piece.EMPTY: ".",
        Piece.WP: "P", Piece.WN: "N", Piece.WB: "B",
        Piece.WR: "R", Piece.WQ: "Q", Piece.WK: "K",
        Piece.BP: "p", Piece.BN: "n", Piece.BB: "b",
        Piece.BR: "r", Piece.BQ: "q", Piece.BK: "k",
    }
    print("\n---------------------------------")
    for rank in range(7, -1, -1):
        print(f"{rank+1} |", end="")
        for file in range(8):
            sq = getSquareIndex(rank, file)
            print(f" {pieceChar[board[sq]]} |", end="")
        print("\n---------------------------------")
    print("    a   b   c   d   e   f   g   h")

# --- KING CHECK ---
def findKing(color):
    king = Piece.WK if color == 1 else Piece.BK
    for i, p in enumerate(board):
        if p == king: return i
    return -1

def isSquareAttacked(square, aggressorColor):
    # Knights
    knightDeltas = [17, 15, 10, 6, -17, -15, -10, -6]
    for d in knightDeltas:
        target = square + d
        if isOnBoard(target):
            if abs(getRank(square)-getRank(target)) + abs(getFile(square)-getFile(target)) in [3]:
                if board[target] == aggressorColor * Piece.WN:
                    return True
    # Kings
    kingDeltas = [8, -8, 1, -1, 9, 7, -9, -7]
    for d in kingDeltas:
        t = square + d
        if isOnBoard(t) and board[t] == aggressorColor * Piece.WK:
            return True
    # Pawns
    if aggressorColor == 1:  # white
        for d in [7, 9]:
            t = square + d
            if isOnBoard(t) and abs(getFile(square)-getFile(t))==1 and board[t] == Piece.WP:
                return True
    else:  # black
        for d in [-7, -9]:
            t = square + d
            if isOnBoard(t) and abs(getFile(square)-getFile(t))==1 and board[t] == Piece.BP:
                return True
    # Sliding
    slidingDeltas = [8,-8,1,-1,9,-9,7,-7]
    for i, d in enumerate(slidingDeltas):
        t = square+d
        while isOnBoard(t):
            if abs(getFile(t)-getFile(t-d)) > 1 and d in [1,-1]: break
            p = board[t]
            if p != Piece.EMPTY:
                if p * aggressorColor > 0:
                    if i<4 and abs(p) in [Piece.WR, Piece.WQ]: return True
                    if i>=4 and abs(p) in [Piece.WB, Piece.WQ]: return True
                break
            t += d
    return False

def isKingInCheck(color):
    k = findKing(color)
    return isSquareAttacked(k, -color)

# --- MAKE / UNDO MOVE ---
def makeMove(move):
    global turn
    board[move.toSquare] = board[move.fromSquare]
    board[move.fromSquare] = Piece.EMPTY
    turn *= -1

def undoMove(move):
    global turn
    board[move.fromSquare] = board[move.toSquare]
    board[move.toSquare] = move.capturedPiece
    turn *= -1

# --- MOVE GENERATION ---
def generateLegalMoves():
    moves = []
    pc = turn
    for fromSq, piece in enumerate(board):
        if piece == Piece.EMPTY or piece*pc <= 0: continue

        def addMove(toSq):
            m = Move(fromSq, toSq, board[toSq])
            makeMove(m)
            if not isKingInCheck(pc): moves.append(m)
            undoMove(m)

        pt = abs(piece)
        if pt == Piece.WP:  # Pawn
            direction = 8 if pc==1 else -8
            startRank = 1 if pc==1 else 6
            if isOnBoard(fromSq+direction) and board[fromSq+direction]==Piece.EMPTY:
                addMove(fromSq+direction)
                if getRank(fromSq)==startRank and board[fromSq+2*direction]==Piece.EMPTY:
                    addMove(fromSq+2*direction)
            for d in [direction-1, direction+1]:
                if isOnBoard(fromSq+d) and abs(getFile(fromSq)-getFile(fromSq+d))==1:
                    if isOpponent(pc, fromSq+d): addMove(fromSq+d)

        elif pt == Piece.WN:  # Knight
            for d in [17,15,10,6,-17,-15,-10,-6]:
                t=fromSq+d
                if isOnBoard(t) and abs(getFile(fromSq)-getFile(t))<=2:
                    if not isSameColor(pc,t): addMove(t)

        else:  # Sliding and King
            dirs=[8,-8,1,-1,9,-9,7,-7]
            start=0; end=8
            if pt==Piece.WR: start,end=0,4
            elif pt==Piece.WB: start,end=4,8
            for i in range(start,end):
                for n in range(1,8):
                    t=fromSq+dirs[i]*n
                    if not isOnBoard(t): break
                    if abs(getFile(t)-getFile(fromSq+dirs[i]*(n-1)))>1: break
                    if isSameColor(pc,t): break
                    addMove(t)
                    if isOpponent(pc,t) or pt==Piece.WK: break
                if pt==Piece.WK: break
    return moves

# --- EVALUATE ---
def evaluateBoard():
    score=0
    for p in board:
        if p!=Piece.EMPTY:
            v=pieceValue(p)
            score += v if p>0 else -v
    return score

# --- ENGINE MOVE ---
def findBestMove_600ELO():
    moves=generateLegalMoves()
    if not moves: return Move(-1,-1)

    bestScore=-200000
    bestMove=moves[0]
    pc=turn
    for m in moves:
        makeMove(m)
        score=pc*evaluateBoard()
        undoMove(m)
        if score>bestScore:
            bestScore=score; bestMove=m

    if random.randint(1,100)<=10:
        nonBest=[m for m in moves if makeMove(m) or (undoMove(m) or pc*evaluateBoard()<bestScore)]
        if nonBest:
            print("\n[ENGINE] Blunder! Random suboptimal move.")
            return random.choice(nonBest)
    print("\n[ENGINE] Best move chosen (70%).")
    return bestMove

# --- USER MOVE PARSER ---
def parseMove(s):
    if len(s)!=4: return Move(-1,-1)
    ff,fr,tf,tr = s[0],s[1],s[2],s[3]
    fromFile,fromRank=ord(ff)-97,int(fr)-1
    toFile,toRank=ord(tf)-97,int(tr)-1
    if not(0<=fromFile<=7 and 0<=toFile<=7 and 0<=fromRank<=7 and 0<=toRank<=7):
        return Move(-1,-1)
    return Move(getSquareIndex(fromRank,fromFile), getSquareIndex(toRank,toFile))

# --- MAIN LOOP ---
def main():
    global turn
    print("--- Simple Python Chess Engine ---")
    print("You are White. Engine is Black.")
    print("Enter moves like e2e4.")
    printBoard()
    while True:
        legalMoves=generateLegalMoves()
        if not legalMoves:
            if isKingInCheck(turn):
                print("\nCHECKMATE!", "Black wins." if turn==1 else "White wins.")
            else: print("\nSTALEMATE! Draw.")
            break
        if turn==1:
            moveStr=input("\nYour move: ")
            userMove=parseMove(moveStr)
            if any(m.fromSquare==userMove.fromSquare and m.toSquare==userMove.toSquare for m in legalMoves):
                chosen=[m for m in legalMoves if m.fromSquare==userMove.fromSquare and m.toSquare==userMove.toSquare][0]
            else:
                print("!!! ILLEGAL MOVE !!! Try again.")
                continue
        else:
            print("\nEngine is thinking...")
            time.sleep(1)
            chosen=findBestMove_600ELO()
            print(f"Engine moves: {chr(getFile(chosen.fromSquare)+97)}{getRank(chosen.fromSquare)+1}"
                  f" to {chr(getFile(chosen.toSquare)+97)}{getRank(chosen.toSquare)+1}")
        makeMove(chosen)
        printBoard()

if __name__=="__main__":
    main()


--- Simple Python Chess Engine ---
You are White. Engine is Black.
Enter moves like e2e4.

---------------------------------
8 | r | n | b | q | k | b | n | r |
---------------------------------
7 | p | p | p | p | p | p | p | p |
---------------------------------
6 | . | . | . | . | . | . | . | . |
---------------------------------
5 | . | . | . | . | . | . | . | . |
---------------------------------
4 | . | . | . | . | . | . | . | . |
---------------------------------
3 | . | . | . | . | . | . | . | . |
---------------------------------
2 | P | P | P | P | P | P | P | P |
---------------------------------
1 | R | N | B | Q | K | B | N | R |
---------------------------------
    a   b   c   d   e   f   g   h



Your move:  d2d4



---------------------------------
8 | r | n | b | q | k | b | n | r |
---------------------------------
7 | p | p | p | p | p | p | p | p |
---------------------------------
6 | . | . | . | . | . | . | . | . |
---------------------------------
5 | . | . | . | . | . | . | . | . |
---------------------------------
4 | . | . | . | P | . | . | . | . |
---------------------------------
3 | . | . | . | . | . | . | . | . |
---------------------------------
2 | P | P | P | . | P | P | P | P |
---------------------------------
1 | R | N | B | Q | K | B | N | R |
---------------------------------
    a   b   c   d   e   f   g   h

Engine is thinking...

[ENGINE] Best move chosen (70%).
Engine moves: a7 to a6

---------------------------------
8 | r | n | b | q | k | b | n | r |
---------------------------------
7 | . | p | p | p | p | p | p | p |
---------------------------------
6 | p | . | . | . | . | . | . | . |
---------------------------------
5 | . | . | . | . | . | . | . | . |
----------


Your move:  c1c4


!!! ILLEGAL MOVE !!! Try again.



Your move:  c1f4



---------------------------------
8 | r | n | b | q | k | b | n | r |
---------------------------------
7 | . | p | p | p | p | p | p | p |
---------------------------------
6 | p | . | . | . | . | . | . | . |
---------------------------------
5 | . | . | . | . | . | . | . | . |
---------------------------------
4 | . | . | . | P | . | B | . | . |
---------------------------------
3 | . | . | . | . | . | . | . | . |
---------------------------------
2 | P | P | P | . | P | P | P | P |
---------------------------------
1 | R | N | . | Q | K | B | N | R |
---------------------------------
    a   b   c   d   e   f   g   h

Engine is thinking...

[ENGINE] Best move chosen (70%).
Engine moves: a6 to a5

---------------------------------
8 | r | n | b | q | k | b | n | r |
---------------------------------
7 | . | p | p | p | p | p | p | p |
---------------------------------
6 | . | . | . | . | . | . | . | . |
---------------------------------
5 | p | . | . | . | . | . | . | . |
----------


Your move:  e2e3



---------------------------------
8 | r | n | b | q | k | b | n | r |
---------------------------------
7 | . | p | p | p | p | p | p | p |
---------------------------------
6 | . | . | . | . | . | . | . | . |
---------------------------------
5 | p | . | . | . | . | . | . | . |
---------------------------------
4 | . | . | . | P | . | B | . | . |
---------------------------------
3 | . | . | . | . | P | . | . | . |
---------------------------------
2 | P | P | P | . | . | P | P | P |
---------------------------------
1 | R | N | . | Q | K | B | N | R |
---------------------------------
    a   b   c   d   e   f   g   h

Engine is thinking...

[ENGINE] Best move chosen (70%).
Engine moves: a5 to a4

---------------------------------
8 | r | n | b | q | k | b | n | r |
---------------------------------
7 | . | p | p | p | p | p | p | p |
---------------------------------
6 | . | . | . | . | . | . | . | . |
---------------------------------
5 | . | . | . | . | . | . | . | . |
----------